<a href="https://colab.research.google.com/github/poojan14/AML-Practicals/blob/master/Practical_8_CCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduced by Harold Hotelling, CCA is a way of inferring information from cross-covariance matrices. Typical use for CCA in the experimental context is to take two sets of variables and see what is common among the two sets. 

In [18]:
!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id':'1bKPBNFvcO5pzfLkinyU7PIkX_jxcEf96'})
download.GetContentFile('NYC_school.csv')

In [0]:
import pandas as pd
df = pd.read_csv('NYC_school.csv')

In [21]:
df.head()

,Adjusted Grade,New?,Other Location Code in LCGMS,School Name,SED Code,Location Code,District,Latitude,Longitude,Address (Full),City,Zip,Grades,Grade Low,Grade High,Community School?,Economic Need Index,School Income Estimate,Percent ELL,Percent Asian,Percent Black,Percent Hispanic,Percent Black / Hispanic,Percent White,Student Attendance Rate,Percent of Students Chronically Absent,Rigorous Instruction %,Rigorous Instruction Rating,Collaborative Teachers %,Collaborative Teachers Rating,Supportive Environment %,Supportive Environment Rating,Effective School Leadership %,Effective School Leadership Rating,Strong Family-Community Ties %,Strong Family-Community Ties Rating,Trust %,Trust Rating,Student Achievement Rating,Average ELA Proficiency,...,Grade 7 ELA - All Students Tested,Grade 7 ELA 4s - All Students,Grade 7 ELA 4s - American Indian or Alaska Native,Grade 7 ELA 4s - Black or African American,Grade 7 ELA 4s - Hispanic or Latino,Grade 7 ELA 4s - Asian or Pacific Islander,Grade 7 ELA 4s - White,Grade 7 ELA 4s - Multiracial,Grade 7 ELA 4s - Limited English Proficient,Grade 7 ELA 4s - Economically Disadvantaged,Grade 7 Math - All Students Tested,Grade 7 Math 4s - All Students,Grade 7 Math 4s - American Indian or Alaska Native,Grade 7 Math 4s - Black or African American,Grade 7 Math 4s - Hispanic or Latino,Grade 7 Math 4s - Asian or Pacific Islander,Grade 7 Math 4s - White,Grade 7 Math 4s - Multiracial,Grade 7 Math 4s - Limited English Proficient,Grade 7 Math 4s - Economically Disadvantaged,Grade 8 ELA - All Students Tested,Grade 8 ELA 4s - All Students,Grade 8 ELA 4s - American Indian or Alaska Native,Grade 8 ELA 4s - Black or African American,Grade 8 ELA 4s - Hispanic or Latino,Grade 8 ELA 4s - Asian or Pacific Islander,Grade 8 ELA 4s - White,Grade 8 ELA 4s - Multiracial,Grade 8 ELA 4s - Limited English Proficient,Grade 8 ELA 4s - Economically Disadvantaged,Grade 8 Math - All Students Tested,Grade 8 Math 4s - All Students,Grade 8 Math 4s - American Indian or Alaska Native,Grade 8 Math 4s - Black or African American,Grade 8 Math 4s - Hispanic or Latino,Grade 8 Math 4s - Asian or Pacific Islander,Grade 8 Math 4s - White,Grade 8 Math 4s - Multiracial,Grade 8 Math 4s - Limited English Proficient,Grade 8 Math 4s - Economically Disadvantaged
0,NaN,NaN,NaN,P.S. 015 ROBERTO CLEMENTE,310100010015,01M015,1,40.721834,-73.978766,"333 E 4TH ST NEW YORK, NY 10009",NEW YORK,10009,"PK,0K,01,02,03,04,05",PK,05,Yes,0.919,"$31,141.72",9%,5%,32%,60%,92%,1%,94%,18%,89%,Meeting Target,94%,Meeting Target,86%,Exceeding Target,91%,Exceeding Target,85%,Meeting Target,94%,Exceeding Target,Approaching Target,2.14,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,NaN,NaN,NaN,P.S. 019 ASHER LEVY,310100010019,01M019,1,40.729892,-73.984231,"185 1ST AVE NEW YORK, NY 10003",NEW YORK,10003,"PK,0K,01,02,03,04,05",PK,05,No,0.641,"$56,462.88",5%,10%,20%,63%,83%,6%,92%,30%,96%,NaN,96%,NaN,97%,NaN,90%,Exceeding Target,86%,Meeting Target,94%,Meeting Target,Exceeding Target,2.63,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,NaN,NaN,NaN,P.S. 020 ANNA SILVER,310100010020,01M020,1,40.721274,-73.986315,"166 ESSEX ST NEW YORK, NY 10002",NEW YORK,10002,"PK,0K,01,02,03,04,05",PK,05,No,0.744,"$44,342.61",15%,35%,8%,49%,57%,4%,94%,20%,87%,Meeting Target,77%,Meeting Target,82%,Approaching Target,61%,Not Meeting Target,80%,Approaching Target,79%,Not Meeting Target,Approaching Target,2.39,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,NaN,NaN,NaN,P.S. 034 FRANKLIN D. ROOSEVELT,310100010034,01M034,1,40.726147,-73.975043,"730 E 12TH ST NEW YORK, NY 10009",NEW YORK,10009,"PK,0K,01,02,03,04,05,06,07,08",PK,08,No,0.860,"$31,454.00",7%,5%,29%,63%,92%,4%,92%,28%,85%,Approaching Target,78%,Meeting Target,82%,Meeting Target,73%,Approaching Target,89%,Meeting Target,88%,Meeting Target,Exceeding Target,2.48,...,55,4,0,0,3,0,0,0,0,0,55,3,0,0,3,0,0,0,0,0,47,1,0,0,0,0,0,0,0,0,48

In [0]:
# choose relevant features
df = df[['Rigorous Instruction %',
      'Collaborative Teachers %',
     'Supportive Environment %',
       'Effective School Leadership %',
   'Strong Family-Community Ties %',
    'Trust %','Average ELA Proficiency',
       'Average Math Proficiency']]
# drop missing values
df = df.dropna()
# separate X and Y groups
X = df[['Rigorous Instruction %',
      'Collaborative Teachers %',
     'Supportive Environment %',
       'Effective School Leadership %',
   'Strong Family-Community Ties %',
    'Trust %'
      ]]
Y = df[['Average ELA Proficiency',
       'Average Math Proficiency']]


In [23]:
for col in X.columns:
    X[col] = X[col].str.strip('%')
    X[col] = X[col].astype('int')
# Standardise the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=True, with_std=True)
X_sc = sc.fit_transform(X)
Y_sc = sc.fit_transform(Y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
!pip install pyrcca

**Canonical Correlation**

In [27]:
import rcca

nComponents = 2 # min(p,q) components
cca = rcca.CCA(kernelcca = False, reg = 0., numCC = nComponents,)
# train on data
cca.train([X_sc, Y_sc])
print('Canonical Correlation Per Component Pair:',cca.cancorrs)
print('% Shared Variance:',cca.cancorrs**2)

Training CCA, kernel = None, regularization = 0.0000, 2 components
Canonical Correlation Per Component Pair: [0.46059902 0.18447786]
% Shared Variance: [0.21215146 0.03403208]


**Canonical Weights**

In [28]:
cca.ws


[array([[-0.00375779,  0.0078263 ],
        [ 0.00061439, -0.00357358],
        [-0.02054012, -0.0083491 ],
        [-0.01252477,  0.02976148],
        [ 0.00046503, -0.00905069],
        [ 0.01415084, -0.01264106]]), array([[ 0.00632283,  0.05721601],
        [-0.02606459, -0.05132531]])]

**Canonical Loadings**

In [30]:
import numpy as np

print('Loading for Math Score:',np.corrcoef(cca.comps[0][:,0],Y_sc[:,0])[0,1])
print('Loading for ELA Score:',np.corrcoef(cca.comps[0][:,0],Y_sc[:,1])[0,1])


Loading for Math Score: -0.4106778140971078
Loading for ELA Score: -0.4578120954218724


**Canonical Covariates**

In [31]:
cca.comps[0]


array([[ 0.00521903,  0.01840999],
       [-0.03173083,  0.00669952],
       [ 0.02066659, -0.02319188],
       ...,
       [ 0.00029259,  0.0018127 ],
       [-0.02341333, -0.00532078],
       [ 0.00647899,  0.00178834]])

In [32]:
cca.comps[0][:,0]


array([ 0.00521903, -0.03173083,  0.02066659, ...,  0.00029259,
       -0.02341333,  0.00647899])

In [33]:
cca.comps[0][:,1]


array([ 0.01840999,  0.00669952, -0.02319188, ...,  0.0018127 ,
       -0.00532078,  0.00178834])

In [34]:
cca.comps[1]


array([[ 0.02079591, -0.00760554],
       [-0.01557288, -0.01886755],
       [ 0.00463834, -0.00862955],
       ...,
       [-0.01923392, -0.00883273],
       [-0.03327623, -0.05742338],
       [-0.02529823, -0.02446945]])

In [35]:
cca.comps[1][:,0]


array([ 0.02079591, -0.01557288,  0.00463834, ..., -0.01923392,
       -0.03327623, -0.02529823])

In [36]:
cca.comps[1][:,1]


array([-0.00760554, -0.01886755, -0.00862955, ..., -0.00883273,
       -0.05742338, -0.02446945])